In [43]:
import numpy as np
import random
import copy
import tqdm.auto
from tqdm.auto import tqdm

## load problem

In [22]:
problem_name = 'problem_4.npz'
problem = np.load(f'projectData/{problem_name}')
problem_x = problem['x']
problem_y = problem['y']

#print(x)
#print(y)

print(problem_x.shape)
print(problem_y.shape)

problem_x_min = np.min(problem_x)
problem_x_max = np.max(problem_x)
print("X_MIN ", problem_x_min , "- X_MAX " ,problem_x_max)

(2, 5000)
(5000,)
X_MIN  -4.999705785571271 - X_MAX  4.999534886222849


### Mapping numpy math functions in a dictionary

In [42]:
import inspect

# Crea un dizionario vuoto
numpy_functions = {}

# Itera su tutti gli attributi del modulo numpy
for name, obj in inspect.getmembers(np):
    # Verifica se l'attributo è una funzione e se è un istanza di ufunc
    if callable(obj) and isinstance(obj, np.ufunc):
        numpy_functions[name] = obj

    
considerated = ["add", "subtract", "multiply", "divide",  "power", "sin", "cos", "log"]
'''considerated = ['sin', 'cos', 'tan', 'arcsin', 'asin', 'arccos', 'acos', 'arctan', 'atan', 'hypot', 'arctan2', 'atan2',
                'degrees', 'radians', 'sinh', 'cosh', 'tanh', 'arcsinh', 'asinh', 'arccosh', 'acosh', 'arctanh', 'atanh',
                'round', 'around', 'rint', 'fix', 'floor', 'ceil', 'trunc', 'log', 'log10', 'log2', 'log1p', 'sinc',
                'add', 'reciprocal', 'positive', 'negative', 'multiply', 'divide', 'power', 'pow', 'subtract',
                'true_divide', 'floor_divide', 'float_power', 'fmod', 'mod', 'remainder']'''

math_functions = { key: value for key, value in numpy_functions.items() if key in considerated}
for m in math_functions.values():
    if m.nout !=1:
        print(m)



print(f"# math functions in numpy: {len(numpy_functions)}")
print("functions:", list(numpy_functions.keys()))



# math functions in numpy: 104
functions: ['abs', 'absolute', 'acos', 'acosh', 'add', 'arccos', 'arccosh', 'arcsin', 'arcsinh', 'arctan', 'arctan2', 'arctanh', 'asin', 'asinh', 'atan', 'atan2', 'atanh', 'bitwise_and', 'bitwise_count', 'bitwise_invert', 'bitwise_left_shift', 'bitwise_not', 'bitwise_or', 'bitwise_right_shift', 'bitwise_xor', 'cbrt', 'ceil', 'conj', 'conjugate', 'copysign', 'cos', 'cosh', 'deg2rad', 'degrees', 'divide', 'divmod', 'equal', 'exp', 'exp2', 'expm1', 'fabs', 'float_power', 'floor', 'floor_divide', 'fmax', 'fmin', 'fmod', 'frexp', 'gcd', 'greater', 'greater_equal', 'heaviside', 'hypot', 'invert', 'isfinite', 'isinf', 'isnan', 'isnat', 'lcm', 'ldexp', 'left_shift', 'less', 'less_equal', 'log', 'log10', 'log1p', 'log2', 'logaddexp', 'logaddexp2', 'logical_and', 'logical_not', 'logical_or', 'logical_xor', 'matmul', 'maximum', 'minimum', 'mod', 'modf', 'multiply', 'negative', 'nextafter', 'not_equal', 'positive', 'pow', 'power', 'rad2deg', 'radians', 'reciprocal', 

In [24]:
'''#vediamo quali funzioni sono compatibili
da_togliere = []
a = 1.1
b = 2.2

for nome, func in numpy_functions.items():
    
    if func.nin == 1:
        try:
            res = func(a)
            if np.isnan(res) or isinstance(res,np.bool):
                #print ("togli ", nome)
                da_togliere.append(nome)
        except:
            #print ("togli ", nome)
            da_togliere.append(nome)


    else:
        try:
            res = func(a,b)
            if np.isnan(res) or isinstance(res,np.bool):
                #print ("togli", nome)
                da_togliere.append(nome)
        except:
            #print ("togli", nome)
            da_togliere.append(nome)

math_functions = { key: value for key, value in numpy_functions.items() if key not in da_togliere}
print(math_functions)           
'''

'#vediamo quali funzioni sono compatibili\nda_togliere = []\na = 1.1\nb = 2.2\n\nfor nome, func in numpy_functions.items():\n    \n    if func.nin == 1:\n        try:\n            res = func(a)\n            if np.isnan(res) or isinstance(res,np.bool):\n                #print ("togli ", nome)\n                da_togliere.append(nome)\n        except:\n            #print ("togli ", nome)\n            da_togliere.append(nome)\n\n\n    else:\n        try:\n            res = func(a,b)\n            if np.isnan(res) or isinstance(res,np.bool):\n                #print ("togli", nome)\n                da_togliere.append(nome)\n        except:\n            #print ("togli", nome)\n            da_togliere.append(nome)\n\nmath_functions = { key: value for key, value in numpy_functions.items() if key not in da_togliere}\nprint(math_functions)           \n'

In [25]:
'''for nome, func in math_functions.items():

    if func.nin == 1:
        res = func(a)
        if res == True:
            print(nome)
    else:
        res = func(a,b)
        if res == True:
            print(nome)'''

'for nome, func in math_functions.items():\n\n    if func.nin == 1:\n        res = func(a)\n        if res == True:\n            print(nome)\n    else:\n        res = func(a,b)\n        if res == True:\n            print(nome)'

### Define node e functions

In [26]:
class Node:
    def __init__(self, operation = None, val = None, left_child = None, right_child = None):
        self.operation = operation
        self.val = val
        self.left_child = left_child
        self.right_child = right_child
        self.fitness = None            #serve solo per il root node
    #IS
    def is_val(self):
        if self.operation is None and self.val is None:  #entrambi None, in teoria non è possibile
            return "not inizialized"
        else:
            if self.val is not None:  #solo operation None, val setted
                return True
        return False
    
    def is_const(self):     #se è una stringa allora è una variabile
        if isinstance(self.val, str):
            return False
        return True
    
    #VAL FUNCTIONS
    def get_val(self):
        return self.val
    
    def set_val(self, val):
        self.val = val

    #OPERATOR FUNCTIONS
    def get_op(self):
        return math_functions[self.operation]  #ritorna la funzione, non il suo nome  
    
    def get_op_name(self):               #serve nella navigate tree per stampare l'albero
        return self.operation            #ritorna il nome dell'operation
    
    def set_op(self, op_name):
        self.operation = op_name

    #CHILD FUNCTIONS
    def get_right_child(self):
        return self.right_child
    
    def get_left_child(self):
        return self.left_child
    
    def set_right_child(self, child):
        self.right_child = child
    
    def set_left_child(self, child):
        self.left_child = child
    
    #FITNESS FUNCTIONS
    def set_fitness(self, fit):
        self.fitness = fit

    def get_fitness(self):
        return self.fitness


    def get_name(self):
        if self.is_val():
            return self.get_val()
        else:
            return self.get_op_name()

### Generate tree

In [27]:
def create_tree(depth = 3, var = {}):  #depth = 3 come minimo per avere una formula del tipo x[0] + np.sin(x[1]) / 5
                           #var è un dizionario con i dati x del problema ( x[0], x[1] .. ecc), la posso dichiarare anche come variabile globale o come vettore
    if depth == 0:         #siamo alla fine, creiamo un nodo val
        if np.random.random()>0.5:
            return Node(val = np.random.uniform(problem_x_min, problem_x_max))  #nodo con le costanti
            
        else:
            return Node(val = np.random.choice(list(var.keys()))) #nodo con i dati x, quando calcoleremo il fitness dalla key risaliamo al value
    
    #altrimenti creamo un nodo con un operazione
    num_param = 0
    while num_param != 2:   #essendo un albero completo, il numero di parametri in ingresso deve essere 2
        operation = np.random.choice(list(math_functions.keys()))
        num_param = math_functions[operation].nin
    left = create_tree(depth-1, var)
    right = create_tree(depth-1, var)
    
    return Node(operation=operation, left_child=left, right_child=right)


def create_grow_tree(depth = 3, var = {}):  
    if depth == 0 or np.random.random()<0.3:  #con probabilità 30% creiamo un nodo val anche se non siamo alla fine     
        if np.random.random()>0.5:
            return Node(val = np.random.uniform(problem_x_min, problem_x_max))             
        else:
            return Node(val = np.random.choice(list(var.keys()))) 
     
    #altrimenti creiamo un nodo con un operazione
    operation = np.random.choice(list(math_functions.keys()))
    #numero di parametri che prende la funzione
    num_param = math_functions[operation].nin   #num parametri in ingresso
    left = create_grow_tree(depth-1, var)
    if num_param == 2:
        right = create_grow_tree(depth-1, var)
    else:
        right = None
    return Node(operation=operation, left_child=left, right_child=right)



### Compute y function and fitness function

In [28]:
def compute_y(node, inputs):   #inputs sono i valori di x, come dizionario 

    if node.is_val():          # è una foglia, con un numero o una variabile
        if node.is_const():   #è una costante
            #print(node.get_val())
            return node.get_val()
        else:                     #è una variabile 
            #print(inputs[node.get_val()])       
            return inputs[node.get_val()]

    else:                       # è un nodo con un operazione
        op = node.get_op()
        
        if node.get_right_child() is not None and node.get_left_child() is not None:
            l = compute_y(node.get_left_child(), inputs)
            r = compute_y(node.get_right_child(), inputs)
            res = op(l,r)
            #print(f'{op} of {l} and {r} = {res}')
            return res
        else:
            if node.get_left_child() is not None:
                l = compute_y(node.get_left_child(), inputs)
                res = op(l)
                #print(f'{op} of {l} = {res}')
                return res
            else:
                r = compute_y(node.get_right_child(), inputs)
                res = op(r)
                #print(f'{op} of {r} = {res}')
                return res
            
    
def fitness_MSE(root_node):
    list_computed_y =np.array([])
    for row in problem_x.T:                                               #el= [x1, x2, x3...] in una certa riga di x 
        var = { f"x{i}": row[i].item() for i in range(len(row))}          #dict = "x1" : x1; "x2": x2 ecc... 
        computed_y = compute_y(root_node, var)
        
        if np.isnan(computed_y):
            computed_y = np.inf     #massimo errore 
        list_computed_y= np.append(list_computed_y, computed_y)

    error = 100*np.square(problem_y-list_computed_y).sum()/len(problem_x)
    return error



### Printing and navigating tree


In [29]:
def navigate_tree(node, depth = 0): 
    if node.is_val():          # è una foglia, con un numero o una variabile
        print(f"depth: {depth}  {str(node.get_val())}")
       
    else:                       # è un nodo con un operazione
        op = node.get_op_name()

        if node.get_left_child() is not None and node.get_right_child() is not None:
            print(f"depth: {depth} {op}")
            navigate_tree(node.get_left_child(), depth +1)
            navigate_tree(node.get_right_child(), depth +1)
        else:
            if node.get_left_child() is not None:
                print(f"depth: {depth} {op}")
                navigate_tree(node.get_left_child(), depth +1)
            else:
                print(f"depth: {depth} {op}")
                navigate_tree(node.get_right_child(), depth +1) 

def node_list(node):   #ritorna la lista di nodi
    if node.is_val():   
        return [node]       
    else:                       # è un nodo con un operazione      
        if node.get_left_child() is not None and node.get_right_child() is not None:
            temp = [node]
            temp.extend(node_list(node.get_left_child()))
            temp.extend(node_list(node.get_right_child()))
            return temp
        
        elif node.get_left_child() is not None and node.get_right_child() is None:
            temp = [node]
            temp.extend(node_list(node.get_left_child()))
            return temp
        
        elif node.get_left_child() is None and node.get_right_child() is not None:
            temp = [node]
            temp.extend(node_list(node.get_right_child()))
            return temp
        
    
    print("errore")
            

            
def get_parent_node(node, list):

    #list = node_list(tree)

    for n in list:
        if n.get_left_child() == node:
            return (n, "left")
        if n.get_right_child() == node:
            return (n, "right")

    #print('parent non trovato')
    return (None, None)


def print_tree_to_string(node): 
    if node.is_val():          # è una foglia, con un numero o una variabile
        return f"{str(node.get_val())}"
       
    else:                       # è un nodo con un operazione
        op = node.get_op_name()

        if node.get_left_child() is not None and node.get_right_child() is not None:
            l = print_tree_to_string(node.get_left_child())
            r = print_tree_to_string(node.get_right_child())
            return f"{op} {l} {r}"
        else:
            if node.get_left_child() is not None:
                l = print_tree_to_string(node.get_left_child())
                return f"{op} {l}"
            else:
                r = print_tree_to_string(node.get_right_child()) 
                return f"{op} {r}"
            

In [30]:
#Funzione che controlla che nella soluzione compaia esattamente una volta x0, una volta x1 ecc
def is_valid(nodelist, var):
    dict = {x: 0 for x in var.keys()}
    for n in nodelist:
        if n.is_val() and not n.is_const():
            dict[n.get_val()]+=1
    for count in dict.values():
        if count != 1:
            return False
    return True  

### Parent selection

In [31]:
def tournament_selection(population, tournament_size = 20):

    candidates = random.sample(population, tournament_size)
    #return max(candidates, key=lambda ind: fitness_MSE(ind))   #o anche con ind.get_fitness()
    #return max(candidates, key=lambda ind: ind.get_fitness())  #o anche con ind.get_fitness()

    '''for i in candidates:
        l = node_list(i)
        if len(l)>15:
            print('candidate lenght', len(l))'''

    sorted_pop = sorted(candidates, key = lambda x: x.get_fitness())

    return sorted_pop[0]

In [32]:
def uniform_selection(pop):
    return random.choice(pop)

### Mutation

In [33]:
#Subtree mutation -> un nodo diventa uno dei suoi sotto nodi
#return false se non è possibile eseguire la mutazione

def mutation_subtree(t):
    tree = copy.deepcopy(t)
    nodeList = node_list(tree)
    count = len(nodeList)
    #print("num nodi : ", count)
    
    if count == 1:
        #print("only one node in the tree")
        return False
       
    avaiableIndexes = []    
    #cerchiamo tra i nodi quali hanno almeno un figlio
    for i in range(1, count):  #non consideriamo il nodo radice  #range non conta l'ultimo valore (si ferma a count-1)
        if nodeList[i].get_left_child() is not None or nodeList[i].get_right_child() is not None:
            avaiableIndexes.append(i)

    if len(avaiableIndexes)<1:
        #print("no avaiable indexes")
        return False        
        
    index = random.choice(avaiableIndexes)
    selected_node = nodeList[index]
    #print(f"selected node index: {index} left child: {selected_node.get_left_child()} right child:  {selected_node.get_right_child()} ")
    #print(f"selected node index: {index}")
    
    parent_node, branch = get_parent_node(selected_node, nodeList)

    if np.random.random()<0.5:   #con prob 50% scegliamo se swappare il nodo con il figlio di dx o di sx, se possibile

        if selected_node.get_left_child() is not None:
            new_node = selected_node.get_left_child()
        else:
            new_node = selected_node.get_right_child()

    else:
        if selected_node.get_right_child() is not None:
            new_node = selected_node.get_right_child()
        else:
            new_node = selected_node.get_left_child()

    #swap
    if branch == "left":
        parent_node.set_left_child(new_node)
        #print(f"swapped node {index} with his left child") 
    else:
        parent_node.set_right_child(new_node)
        #print(f"swapped node {index} with his right child") 

    #aggiorniamo il fitness 
    fit = fitness_MSE(tree)
    tree.set_fitness(fit)
    
    return tree
     

##############################################################################################################

def mutation_point(t, var):  #cambiamo l'operatore o il valore in un punto
    tree = copy.deepcopy(t)
    nodeList = node_list(tree)
    count = len(nodeList)
        
    index = random.randint(0,count-1)  #si può fare con random.choice, index serve per capire quale nodo precisamente
    selected_node = nodeList[index]
    #print(f"selected node index: {index} {selected_node.get_name()}")

    if selected_node.is_val():

        if np.random.random()>0.5:      
            last_val = selected_node.get_val()     #nodo con le costanti
            new_val = last_val
            while new_val == last_val:
                new_val = np.random.uniform(problem_x_min, problem_x_max)
            selected_node.set_val(new_val)
            #print("new val: ", selected_node.get_val())
        else:
            selected_val = selected_node.get_val() 
            while selected_val == selected_node.get_val() and len(var)!=1: #se c'è solo una variabile x0, skippiamo il ciclo
                selected_val = np.random.choice(list(var.keys()))
            selected_node.set_val(selected_val) #nodo con i dati x
        
    else:
        operation = selected_node.get_op_name()
        num_param = math_functions[operation].nin

        #scegliamo una nuova operazione con lo stesso numero di parametri
        new_num_param = 0
        new_operation = operation
        while new_num_param != num_param or new_operation == operation: 
            new_operation = np.random.choice(list(math_functions.keys()))
            new_num_param = math_functions[new_operation].nin
    
        selected_node.set_op(new_operation)

    #aggiorniamo il fitness 
    fit = fitness_MSE(tree)
    tree.set_fitness(fit)
    return tree

##################################################################################################################


def mutation_permutation(t):
    tree = copy.deepcopy(t)
    nodeList = node_list(tree)
    list_avaiable = list(filter(lambda x: (x.is_val()==False and x.get_left_child() is not None and x.get_right_child() is not None), nodeList))
    
    if len(list_avaiable)<1:
        #print("no avaiable indexes")
        return False 

    selected_node = random.choice(list_avaiable)
    #print(f"Avaiable nodes: {len(list_avaiable)} selected node : {selected_node.get_op_name()}")

    #swap dei nodi figli
    r = selected_node.get_right_child()
    l = selected_node.get_left_child()

    selected_node.set_left_child(r)
    selected_node.set_right_child(l)

    #aggiorniamo il fitness 
    fit = fitness_MSE(tree)
    tree.set_fitness(fit)

    return tree

#########################################################################################################

def mutation_hoist(t):

    tree = copy.deepcopy(t)
    nodeList = node_list(tree)
    count = len(nodeList)
    
    '''if count == 1:
        print("only one node in the tree")
        return tree'''
       
    avaiableIndexes = []    
    #cerchiamo tra i nodi quali hanno almeno un figlio
    for i in range(1, count):  #non consideriamo il nodo radice alrtimenti la mutazione non ha effetto  #range non conta l'ultimo valore (si ferma a count-1)
        if nodeList[i].get_left_child() is not None or nodeList[i].get_right_child() is not None:
            avaiableIndexes.append(i)

    if len(avaiableIndexes)<1:
        #print("no avaiable indexes")
        return False        
        
    index = random.choice(avaiableIndexes)
    
    selected_node = nodeList[index]
    #print(f"Avaiable nodes: {len(avaiableIndexes)} selected node : {selected_node.get_op_name()}")    

    #aggiorniamo il fitness 
    fit = fitness_MSE(selected_node)
    selected_node.set_fitness(fit)

    return selected_node

######################################################################################################




### Crossover - Recombination

In [34]:
def xover(t1, t2):     #ritorna due nuovi alberi, ma cambia anche i parents, altrimenti metodo deepcopy
    
    tree1 = copy.deepcopy(t1)
    tree2 = copy.deepcopy(t2)

    nodeList1 = node_list(tree1)
    nodeList2 = node_list(tree2)

    selected_node1 = random.choice(nodeList1)
    selected_node2 = random.choice(nodeList2)

    ########### FOR DEBUG ##########################################

    '''if len(nodeList1)>15:
        print("nodelist1: ", len(nodeList1))
    if len(nodeList2)>15:
        print("nodelist2: ", len(nodeList2))

    if selected_node1.is_val():
        name1 = selected_node1.get_val()
    else:
        name1 =selected_node1.get_op_name()

    if selected_node2.is_val():
        name2 = selected_node2.get_val()
    else:
        name2 =selected_node2.get_op_name()
    
    print(f"selected node 1 : {name1}")
    print(f"selected node 2 : {name2}") '''
    ################################################################

    parent1, branch1 = get_parent_node(selected_node1, nodeList1)
    parent2, branch2 = get_parent_node(selected_node2, nodeList2)

    if parent1 is not None and parent2 is not None:  #nessuno dei due è il nodo radice

        if branch1 == 'left':
            parent1.set_left_child(selected_node2)
        else:
            parent1.set_right_child(selected_node2)

        if branch2 == 'left':
            parent2.set_left_child(selected_node1)
        else:
            parent2.set_right_child(selected_node1)
    
    else: #uno dei due è il nodo radice
        if parent1 is None and parent2 is None: #entrambi nodo radice, la recombination sarebbe inutile
            #print("both nodi radice")
            return (tree1, tree2)
        elif parent1 is None and parent2 is not None:  #selected node 1 è nodo radice
            #print("selected node 1 è nodo radice")
            if branch2 == 'left':
                parent2.set_left_child(selected_node1)
            else:
                parent2.set_right_child(selected_node1)
            tree1 = selected_node2

        elif parent1 is not None and parent2 is None:  #selected node 2 è nodo radice
            #print("selected node 2 è nodo radice")
            if branch1 == 'left':
                parent1.set_left_child(selected_node2)
            else:
                parent1.set_right_child(selected_node2)
            tree2 = selected_node1
        

    #aggiorniamo i fitness 
    fit = fitness_MSE(tree1)
    tree1.set_fitness(fit)

    fit = fitness_MSE(tree2)
    tree2.set_fitness(fit)

    return (tree1, tree2)

### Generate population

In [35]:
#Genera una popolzione di dimensione SIZE, di metà full e metà grow tree
def generate_population(SIZE, depth = 3 ):
    population = []
    population_as_string = []
    i=0
    while(len(population)) < SIZE:
        row = problem_x.T[0]          #row = [x1, x2, x3...] in una riga di x, ci interessa solo il nome di variabili 
        var = { f"x{i}": row[i].item() for i in range(len(row))} 
        if i<(SIZE/4):
            tree = create_tree(depth,var)
        else:
            tree = create_grow_tree(depth,var)

        #controlliamo se non c'è gia un albero uguale
        tree_str = print_tree_to_string(tree)
        if tree_str not in population_as_string:

            population_as_string.append(tree_str)
            
            #calcoliamo il fitness di questo individuo, sè è valido, lo aggiungiamo all popolazione
            fit = fitness_MSE(tree)
            if not np.isnan(fit) and fit != np.inf:
                tree.set_fitness(fit)
                population.append(tree)
                i+=1  

    return population        

## Algoritmo

In [ ]:
numbers = [1,2,3,4,5,6,7,8]
for numb in numbers:

    problem_name = f'problem_{numb}.npz'
    print('*'*30)
    print(problem_name)
    problem = np.load(f'projectData/{problem_name}')
    problem_x = problem['x']
    problem_y = problem['y']

    problem_x_min = np.min(problem_x)
    problem_x_max = np.max(problem_x)

    SIZE = 100
    population = generate_population(SIZE)

    #ordiniamo per fitness la pop
    sorted_pop = sorted(population, key = lambda x: x.get_fitness())

    '''b = sorted_pop[0]
    print(print_tree_to_string(b))'''


        #generational, offspring replace parents

    NUM_STEPS = 100
    mutation_prob = 0.05
    row = problem_x.T[0]          #row = [x1, x2, x3...] in una riga di x, ci interessa solo il numero di variabili 
    var = { f"x{i}": row[i].item() for i in range(len(row))}

    #salviamo la migliore soluzione
    best = sorted_pop[0]
    print("first best: ", print_tree_to_string(best))

    for steps in tqdm(range(NUM_STEPS)):

        #print('step n:', steps)
        new_pop = []

        #genetic operations

        if random.random()<mutation_prob:  #recombination

            while len(new_pop) < SIZE:

                #parent selection
                p1 = tournament_selection(population)
                p2 = tournament_selection(population)

                off1, off2 = xover(p1,p2)   #potremmo anche controllare che i due offspring siano diversi dai parents (caso di entrambi i nodi radice switchati) 
                new_pop.append(off1)
                new_pop.append(off2)

        else:                               #mutation
            while len(new_pop) < SIZE:
                #selezioniamo la mutazione [0: subtree 1: point 2: permutation 3: hoist]
                sel = random.randint(0,3)
                #print("sel: ", sel)
                p1 = tournament_selection(population)

                if sel==0: #subtree
                    off1 = mutation_subtree(p1)
                    if off1:
                        new_pop.append(off1)

                elif sel==1: #point
                    off1 = mutation_point(p1, var)
                    new_pop.append(off1)
            
                elif sel == 2: #permutation
                    off1 = mutation_permutation(p1)
                    if off1:
                        new_pop.append(off1)

                elif sel == 3: #hoist
                    off1 = mutation_hoist(p1)
                    if off1:
                        new_pop.append(off1)
    
        #ora abbiamo la nuova popolazione
        sorted_pop = sorted(new_pop, key = lambda x: x.get_fitness())

        if sorted_pop[0].get_fitness() < best.get_fitness():
            #print('new_best: ', sorted_pop[0].get_fitness())
            #print('old_best: ', best.get_fitness())
            best = sorted_pop[0]
            #print('new_best_setted: ', best.get_fitness())

        population = copy.deepcopy(new_pop)

        new_pop = []

    for p in population:
        l = node_list(p)
        if is_valid(l, var):
            print("valid solution: ")
            print(print_tree_to_string(p))
            print(fitness_MSE(p))
            break

    print("best solution: ")
    print(problem_name)
    print(print_tree_to_string(best))
    print(fitness_MSE(best))
    print()
    print('#'*30)
    
    